##### 0. Imports

In [22]:
import neptune as neptune
from dotenv import load_dotenv

import os

import sys
sys.path.insert(0, os.path.abspath('../'))
from src.data.make_dataset import Data
from src.models.train_model import Trainer
from src.utils.utils import get_config, get_previous_month

##### 1. Load environement variables & start a Neptune run

In [23]:
load_dotenv()

PROJECT_NAME = os.getenv('PROJECT_NAME')
NEPTUNE_API_TOKEN = os.getenv('NEPTUNE_API_TOKEN')
BASE_URL = os.getenv('BASE_URL')

In [24]:
run = neptune.init_run(project=PROJECT_NAME, api_token=NEPTUNE_API_TOKEN, capture_hardware_metrics=True)

https://app.neptune.ai/hicham.benbriqa/mlops-zoom-camp/e/MLOPS-22


##### 2. Download and prepare the train and test data 

In [25]:
## Get the year and month of the data to be used for training
## Get the previous year and previous month of the data to be used for testing
taxi_type, train_year, train_month = get_config(config_type='data')
test_year, test_month = get_previous_month(train_year, train_month)

## Get the hyperparameters for the model
params = get_config(config_type='model')

# set tags for organization purposes
run["sys/tags"].add([taxi_type])
run["sys/tags"].add([str(train_year)])
run["sys/tags"].add([str(train_month)])

In [26]:
print(f'Taxi type: {taxi_type}')
print(f'Train year: {train_year}')
print(f'Train month: {train_month}')
print(f'Test year: {test_year}')
print(f'Test month: {test_month}')

Taxi type: green
Train year: 2022
Train month: 3
Test year: 2022
Test month: 2


In [27]:
## Instantiate a Data object for training and testing
train_data = Data(taxi_type, train_year, train_month, mode='train', root_folder='../data')
test_data = Data(taxi_type, test_year, test_month, mode='test', root_folder='../data')

In [28]:
## Run the Data object to download, prepare and save the train and test data
train_data.run()
test_data.run()

In [29]:
## Get the target values for the train and test data to be used for evaluation
y_train, y_test = train_data.get_target_values(), test_data.get_target_values()

In [30]:
## track train/test data files
run["datasets/train"].track_files(train_data.paths['processed'])
run["datasets/test"].track_files(test_data.paths['processed'])

##### 3. Train and evaluate the pipeline 

In [31]:
## Instantiate a Trainer object to train and evaluate the model
trainer = Trainer(train_data.data_dict, y_train, test_data.data_dict, y_test, params=params, root_folder='../models')
trainer.train()
rmse = trainer.evaluate()
print(trainer.params, rmse)

{'n_estimators': 500, 'max_depth': 40} 5.863639232115829


In [32]:
## Save the pipeline
trainer.save_pipeline()

In [33]:
# track hyperparams
run["parameters"] = trainer.params
# upload rmse
run["test/rmse"].append(rmse)
#upload model artifact 
run["trained_pipelines/pipeline"].upload(trainer.pipeline_path)

In [34]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 8 operations to synchronize with Neptune. Do not kill this process.
Still waiting for the remaining 8 operations (0.00% done). Please wait.
Still waiting for the remaining 8 operations (0.00% done). Please wait.
All 8 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/hicham.benbriqa/mlops-zoom-camp/e/MLOPS-22/metadata


#  Neptune and Optuna for hyperparameter optimazition

In [8]:
run = neptune.init_run(project=PROJECT_NAME, api_token=NEPTUNE_API_TOKEN)

/tmp/ipykernel_399273/1758828451.py:1: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  run = neptune.init_run(project=PROJECT_NAME, api_token=NEPTUNE_API_TOKEN)


https://app.neptune.ai/hicham.benbriqa/mlops-zoom-camp/e/MLOPS-19


In [9]:
def get_data(taxi_type, year, month):
    train_year, train_month = year, month
    test_year, test_month = get_previous_month(train_year, train_month)

    train_data = Data(taxi_type, train_year, train_month, mode='train', root_folder='../data')
    test_data = Data(taxi_type, test_year, test_month, mode='test', root_folder='../data')

    train_data.run()
    test_data.run()

    y_train, y_test = train_data.get_target_values(), test_data.get_target_values()

    return train_data.data_dict, test_data.data_dict, y_train, y_test

def train_pipeline(train_dict, test_dict, y_train, y_test, params):
    trainer = Trainer(dict_train=train_dict, dict_test=test_dict, y_train=y_train,y_test=y_test, params=params, root_folder='../models')
    trainer.train()
    return trainer

def run_pipeline(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 100)
    max_depth = trial.suggest_int('max_depth', 10, 20)
    #max_samples = trial.suggest_float("max_samples", 0.2, 1)

    # Get the specified hyperpaprameters
    params = {'n_estimators': n_estimators, 'max_depth': max_depth}

    ## Get data of the specified taxi type, year and month in The config file. hint: parametrizing the way the data is loaded
    ## can help you extend the tuning to the data used for training and testing.
    taxi_type, year, month = get_config(config_type='data')
    train_dict, test_dict, y_train, y_test = get_data(taxi_type, year, month)
    
    trainer = train_pipeline(train_dict, test_dict, y_train, y_test, params)
    return trainer

def objective(trial):
    trainer =  run_pipeline(trial)
    print('here')
    rmse = trainer.evaluate()
    return rmse

In [10]:
import optuna
import neptune.integrations.optuna as optuna_utils

neptune_callback = optuna_utils.NeptuneCallback(run)

/home/hicham/.pyenv/versions/3.9.7/envs/test_optuna/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
study = optuna.create_study(direction='maximize', study_name="starter-experiment-9", storage='sqlite:///starter.db')

[I 2023-06-26 13:43:54,793] A new study created in RDB with name: starter-experiment-9


In [ ]:
study.optimize(objective, n_trials=10, callbacks=[neptune_callback])

In [ ]:
trial = study.best_trial
print("Best Score: ", trial.value)
print("Best Params: ")
for key, value in trial.params.items():
    print("  {}: {}".format(key, value))

In [ ]:
best_model = create_model(study.best_trial)
best_model.fit(X_train, y_train)
y_pred = rf.predict(X_val)
mse = mean_squared_error(y_val, y_pred, squared=False)
print("Performance: ", mse)

In [ ]:
dump(rf, "best_model.pkl")

# Neptune model registery

The Model object is suitable for storing general metadata that is shared by all versions of the model – for example, the model signature and validation datasets.

In [14]:
# Create a new model if you change the algorithm or train/test data for example, when doing hyperparameter tuning, create a new model version instead
model = neptune.init_model(
    project=PROJECT_NAME, 
    api_token=NEPTUNE_API_TOKEN,
    name="Best RF optuna model",
    key = 'RF1'
)

https://app.neptune.ai/hicham.benbriqa/mlops-zoom-camp/m/MLOPS-RF1


Track model metadata by assigning them to the model object:

In [16]:
model["data/train"].track_files(train_data.paths['processed'])
model["data/test"].track_files(test_data.paths['processed'])


## Creating a model version

Once you have registered a model, you can create and store versions of it. This lets you track the stage, binaries, and relevant metadata of each model version separately.

Initialize a ModelVersion object based on the ID of an existing model:

In [35]:
model_version = neptune.init_model_version(project=PROJECT_NAME, api_token=NEPTUNE_API_TOKEN,
    model="MLOPS-RF1") #Neptune ID of the model, which comes from the project 
                             #key and model key put together. You can find it in the 
                             #leftmost column of the models table.

https://app.neptune.ai/hicham.benbriqa/mlops-zoom-camp/m/MLOPS-RF1/v/MLOPS-RF1-2


In [36]:
model_version["model/binary"].upload("../models/pipeline.joblib")

model_version["model/parameters"] = params
model_version["model/environment/poetry.lock"].upload("../poetry.lock")
model_version["model/environment/pyproject.toml"].upload("../pyproject.toml")
model_version["test/rmse"] = rmse

To stop the connection to Neptune and sync all data, call the stop() method 

In [37]:
model_version.stop()

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 6 operations to synchronize with Neptune. Do not kill this process.
Still waiting for the remaining 6 operations (0.00% done). Please wait.
Still waiting for the remaining 6 operations (0.00% done). Please wait.
All 6 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/hicham.benbriqa/mlops-zoom-camp/m/MLOPS-RF1/v/MLOPS-RF1-2/metadata


In [20]:
model.stop()

Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/hicham.benbriqa/mlops-zoom-camp/m/MLOPS-RF1/metadata


In [21]:
run.stop()

### Manage models version stages

In [39]:
# Get all the versions of the RF-1 model
import neptune

model = neptune.init_model(project=PROJECT_NAME, api_token=NEPTUNE_API_TOKEN, with_id="MLOPS-RF1")

model_versions_df = model.fetch_model_versions_table().to_pandas()

https://app.neptune.ai/hicham.benbriqa/mlops-zoom-camp/m/MLOPS-RF1


In [49]:
model_versions_df.head(3)

,sys/creation_time,sys/id,sys/model_id,sys/modification_time,sys/monitoring_time,sys/owner,sys/ping_time,sys/running_time,sys/size,sys/stage,sys/state,sys/tags,sys/trashed,model/parameters/max_depth,model/parameters/n_estimators,test/rmse
0,2023-07-05 16:59:49.366000+00:00,MLOPS-RF1-2,MLOPS-RF1,2023-07-05 17:03:08.372000+00:00,88,hicham.benbriqa,2023-07-05 17:01:38.332000+00:00,21.469,284641828.0,staging,Inactive,,False,40,500,5.863639
1,2023-07-04 17:03:10.396000+00:00,MLOPS-RF1-1,MLOPS-RF1,2023-07-05 17:03:12.351000+00:00,20,hicham.benbriqa,2023-07-04 17:03:35.479000+00:00,0.000,56772755.0,staging,Inactive,,False,40,100,5.863760


In [46]:
 # Get the version id of the model version that performed the best
 version_id = model_versions_df[model_versions_df['test/rmse']==model_versions_df['test/rmse'].min()]['sys/id'].values[0]

In [47]:
# Set the version to production
model_version = neptune.init_model_version(project=PROJECT_NAME, api_token=NEPTUNE_API_TOKEN,
    with_id=version_id,
)

model_version.change_stage("production")


https://app.neptune.ai/hicham.benbriqa/mlops-zoom-camp/m/MLOPS-RF1/v/MLOPS-RF1-2


In [48]:
model.stop()
model_version.stop()

Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/hicham.benbriqa/mlops-zoom-camp/m/MLOPS-RF1/metadata
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/hicham.benbriqa/mlops-zoom-camp/m/MLOPS-RF1/v/MLOPS-RF1-2/metadata


In [54]:
model = neptune.init_model(
        project=PROJECT_NAME,
        api_token=NEPTUNE_API_TOKEN,
        with_id="MLOPS-RF1",)
model_versions_df = model.fetch_model_versions_table().to_pandas()
production_models = model_versions_df[model_versions_df["sys/stage"] == "production"]

https://app.neptune.ai/hicham.benbriqa/mlops-zoom-camp/m/MLOPS-RF1


In [4]:
model_version["model/binary"]

NameError: name 'model_version' is not defined

In [55]:
production_models

,sys/creation_time,sys/id,sys/model_id,sys/modification_time,sys/monitoring_time,sys/name,sys/owner,sys/ping_time,sys/running_time,sys/size,sys/stage,sys/state,sys/tags,sys/trashed,model/parameters/max_depth,model/parameters/n_estimators,test/rmse
0,2023-07-05 16:59:49.366000+00:00,MLOPS-RF1-2,MLOPS-RF1,2023-07-05 17:51:55.454000+00:00,88,Untitled,hicham.benbriqa,2023-07-05 17:53:07.660000+00:00,358.337,284641859.0,production,Active,,False,40,500,5.863639


In [3]:
from joblib import load
model = load("../models/pipeline.joblib")